In [1]:
from tensorflow.keras.datasets import cifar10

In [5]:
# handle ssl errors
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [6]:
(features_train, label_train), (features_test, label_test) = cifar10.load_data()

170508288/170498071 [==============================] - 69s 0us/step


In [7]:
# print shape of features_train
features_train.shape

(50000, 32, 32, 3)

In [8]:
batch_size, img_height, img_width = 16, 32, 32
print(batch_size, img_height, img_width)

16 32 32


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
train_img_gen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [13]:
val_img_gen = ImageDataGenerator(rescale=1./255)

In [14]:
train_data_gen = train_img_gen.flow(features_train, label_train, batch_size=batch_size)

In [15]:
val_data_gen = val_img_gen.flow(features_test, label_test, batch_size=batch_size)

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [17]:
np.random.seed(8)
tf.random.set_seed(8)

In [19]:
model = tf.keras.Sequential([layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width, 3)), # convolution layer with 64 kernels of shape 3, ReLU as the activation function and the necessary input dimensions
                            layers.MaxPool2D(), # max pooling layer
                            layers.Conv2D(128, 3, activation='relu'), # convolution layer with 128 kernels of shape 3 and ReLU as the activation function
                            layers.MaxPool2D(), # max pooling layer
                            layers.Flatten(), # flatten layer
                            layers.Dense(128, activation='relu'), # fully connected layer with 128 units and ReLU as the activation function
                            layers.Dense(10, activation='softmax')]) # fully connected layer with 10 units and Softmax as the activation function

2022-05-24 06:58:52.712066: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [21]:
model.compile(loss='sparse_categorical_crossentropy', # using sparse_categorical_crossentropy because labels are not encoded.
              optimizer=optimizer,
              metrics=['accuracy'])

In [22]:
model.fit(train_data_gen, steps_per_epoch=len(features_train) // batch_size, 
         epochs=5, validation_data=val_data_gen, 
          validation_steps=len(features_test) // batch_size)

Epoch 1/5
3125/3125 [==============================] - 54s 17ms/step - loss: 1.4759 - accuracy: 0.1043 - val_loss: 1.1425 - val_accuracy: 0.0901
Epoch 2/5
3125/3125 [==============================] - 54s 17ms/step - loss: 1.1597 - accuracy: 0.1003 - val_loss: 1.0546 - val_accuracy: 0.0941
Epoch 3/5
3125/3125 [==============================] - 57s 18ms/step - loss: 1.0574 - accuracy: 0.0997 - val_loss: 0.9442 - val_accuracy: 0.1009
Epoch 4/5
3125/3125 [==============================] - 58s 19ms/step - loss: 0.9935 - accuracy: 0.0997 - val_loss: 0.9059 - val_accuracy: 0.1123
Epoch 5/5
3125/3125 [==============================] - 59s 19ms/step - loss: 0.9434 - accuracy: 0.1014 - val_loss: 0.8501 - val_accuracy: 0.1138


In [24]:
# model 2
model2 = tf.keras.Sequential([layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width, 3)),
                            layers.MaxPool2D(), 
                            layers.Conv2D(128, 3, activation='relu'), 
                            layers.MaxPool2D(), 
                            layers.Conv2D(256, 3, activation='relu'), # added layer
                            layers.Flatten(), 
                            layers.Dense(256, activation='relu'), # added layer
                            layers.Dense(128, activation='relu'), 
                            layers.Dense(10, activation='softmax')])

In [25]:
# compile model2
model2.compile(loss='sparse_categorical_crossentropy', # using sparse_categorical_crossentropy because labels are not encoded.
              optimizer=optimizer,
              metrics=['accuracy'])

In [26]:
# fit model2
model2.fit(train_data_gen, steps_per_epoch=len(features_train) // batch_size, 
         epochs=5, validation_data=val_data_gen, 
          validation_steps=len(features_test) // batch_size)

Epoch 1/5
3125/3125 [==============================] - 68s 22ms/step - loss: 2.3042 - accuracy: 0.1208 - val_loss: 2.3028 - val_accuracy: 1.0000
Epoch 2/5
3125/3125 [==============================] - 68s 22ms/step - loss: 2.3029 - accuracy: 0.1411 - val_loss: 2.3027 - val_accuracy: 1.0000
Epoch 3/5
3125/3125 [==============================] - 74s 24ms/step - loss: 2.3029 - accuracy: 0.1725 - val_loss: 2.3026 - val_accuracy: 0.0000e+00
Epoch 4/5
3125/3125 [==============================] - 74s 24ms/step - loss: 2.3029 - accuracy: 0.0909 - val_loss: 2.3027 - val_accuracy: 0.0000e+00
Epoch 5/5
3125/3125 [==============================] - 75s 24ms/step - loss: 2.3028 - accuracy: 0.1878 - val_loss: 2.3028 - val_accuracy: 0.0000e+00
